In [19]:
from data import Datagenerator, TrainingStats
from copy import deepcopy
from model import LEO, load_model, save_model
from  torch.nn import MSELoss
from easydict import EasyDict as edict
from utils import load_config, check_experiment
from run import train_model
import torch.optim as optim
import argparse
import torch 
import torch.optim
import gc
from utils import display_data_shape, get_named_dict, one_hot_target,\
    softmax, sparse_crossentropy, calc_iou_per_class, log_data, load_config
import matplotlib.pyplot as plt
import numpy as np

## Train

In [20]:
config = load_config()

In [ ]:
leo, meta_data, classes = train_model(config)

num tasks: 7
tr_data shape: torch.Size([7, 2, 384, 512, 14]),tr_data_masks shape: torch.Size([7, 2, 384, 512]), val_data shape: torch.Size([7, 5, 384, 512, 14]),val_masks shape: torch.Size([7, 5, 384, 512])


C:\Users\temil\OneDrive\Documents\Study\UHildesheim\Project\Scripts\leo-srp-project\merge_to_dev\leo_segmentation\leo_segmentation\utils.py:205: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output contiguous tensor even if some of the inputs are in channels_last format. (Triggered internally at  ..\aten\src\ATen\native\TensorIterator.cpp:918.)
  loss = torch.sum(-1 * target * torch.log(pred), dim=channel_dim)


Class: car, Episode: 1, Val IOU: 0.00026729403812896245
Class: bus, Episode: 1, Val IOU: 0.0
Class: aeroplane, Episode: 1, Val IOU: 0.0
Class: bicycle, Episode: 1, Val IOU: 0.0008932753971014225
Class: bottle, Episode: 1, Val IOU: 0.0
Class: chair, Episode: 1, Val IOU: 0.0
Class: cow, Episode: 1, Val IOU: 0.0

mode:meta_train, episode:001, kl_loss:6.920055, total_val_loss:0.692386
Class: cat, Episode: 2, Val IOU: 0.0003486674843634833
Class: aeroplane, Episode: 2, Val IOU: 3.515865342357388e-06
Class: car, Episode: 2, Val IOU: 0.0033367823314670733
Class: bird, Episode: 2, Val IOU: 2.5826446280991735e-05
Class: bicycle, Episode: 2, Val IOU: 0.04137031709115359
Class: chair, Episode: 2, Val IOU: 0.0003426485786047717
Class: bus, Episode: 2, Val IOU: 0.0

mode:meta_train, episode:002, kl_loss:328.662903, total_val_loss:0.693362
Class: cow, Episode: 3, Val IOU: 0.0001122180154225306
Class: aeroplane, Episode: 3, Val IOU: 0.0
Class: bus, Episode: 3, Val IOU: 1.394991978796122e-05
Class: ca

## Predictions on Meta Val / Test

In [ ]:
leo

In [ ]:
dataloader = Datagenerator(config, dataset="pascal_voc", data_type="meta_val")
val_meta_data = dataloader.get_batch_data()
class_in_metadata = val_meta_data[-1]

In [ ]:
class_in_metadata

In [ ]:
select_class = -1
select_sample = 0

In [ ]:
print("class", class_in_metadata[select_class], "is selected")
batch_data = get_named_dict(val_meta_data, select_class)
input_data = batch_data.val_data[select_sample].permute(1, 2, 0)
plt.title("First three channels of embeddings is visualized")
plt.imshow(input_data.cpu().detach().numpy()[:,:,:3])

In [ ]:
batch_data.val_data.shape

In [ ]:
latents, kl_loss = leo.forward_encoder(batch_data.tr_data)
tr_loss, predicted_weights, _ = leo.forward_decoder(batch_data.tr_data, latents, batch_data.tr_data_masks)
val_loss = leo.finetuning_inner_loop(batch_data, tr_loss, predicted_weights)
_, _, predictions = leo.forward_decoder(batch_data.val_data, latents, batch_data.val_data_masks)

In [ ]:
select_sample = 0
def plot_batch_mask(batch_data, select_sample):
    input_mask = batch_data.val_data_masks[select_sample]
    plt.title("ground truth mask")
    plt.imshow(input_mask.cpu().detach().numpy(), cmap="gray")
    return

plot_batch_mask(batch_data, select_sample)

In [ ]:
def plot_prediction_mask(pred_mask, select_sample):
    plt.title("mask prediction")
    plt.imshow(np.argmax(pred_mask[select_sample].cpu().detach().numpy(), 0), cmap="gray")
    return

plot_prediction_mask(predictions, select_sample)

In [ ]:
calc_iou_per_class(predictions, batch_data.val_data_masks)